# Manual LED Onset Annotation

Use this notebook to scrub frames and record the LED-on frame index for each video.
This is independent of `bab_datasets` syncing code.

In [7]:
# !pip install "git+https://github.com/helonayala/bab_datasets.git#egg=bab_datasets[video,notebook]"

In [8]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import bab_datasets as nod

## Video path and limits

In [9]:
# Pick a video name from the dataset registry
video_name = 'multisine_02'

# Download (if needed) and return local path
video_path = nod.load_video_path(video_name)
print('video_path:', video_path)

# Limit to first N seconds for faster browsing (set None for full video)
max_seconds = 10

video_path: /Users/helon/Desktop/neural_ODE/videos_BAB/multisine_02.MOV


In [10]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

max_frames = int(fps * max_seconds) if max_seconds is not None else frame_count
max_frames = min(max_frames, frame_count)

print('fps:', fps, 'frames:', frame_count, 'size:', (width, height))
print('max_frames:', max_frames)

fps: 30.001847404396823 frames: 2436 size: (1920, 1080)
max_frames: 300


## Frame browser

In [11]:
def show_frame(i):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        print('Failed to read frame', i)
        return
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    t = i / fps
    plt.figure(figsize=(7,4))
    plt.imshow(frame_rgb)
    plt.title(f'frame={i}, time={t:.3f}s')
    plt.axis('off')
    plt.show()

slider = widgets.IntSlider(min=0, max=max_frames-1, step=1, value=0, description='frame')
display(widgets.interact(show_frame, i=slider))

interactive(children=(IntSlider(value=0, description='frame', max=299), Output()), _dom_classes=('widget-inter…

<function __main__.show_frame(i)>

In [ ]:
from IPython.display import display, clear_output

# LED_frame = 291 # swept sine
# LED_frame = 415 # rampa positiva
# LED_frame = 266 # rampa negativa
# LED_frame = 273 # random_steps_W
# LED_frame = 375 # random_steps_X
# LED_frame = 238 # random_steps_Y
# LED_frame = 288 # random_steps_Z
# LED_frame = 279 # multisine_01
LED_frame = 260 # multisine_02

frame_input = widgets.IntText(value=LED_frame, description="LED frame")
play_button = widgets.Button(description="Play ±5 frames", button_style="info")
out = widgets.Output()

def show_frame(i):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        print("Failed to read frame", i)
        return
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    t = i / fps
    plt.figure(figsize=(7,4))
    plt.imshow(frame_rgb)
    plt.title(f"frame={i}, time={t:.3f}s")
    plt.axis("off")
    plt.show()

def on_play(_):
    with out:
        clear_output(wait=True)
        i0 = frame_input.value
        for i in range(i0 - 5, i0 + 6):
            if i < 0 or i >= frame_count:
                continue
            show_frame(i)

play_button.on_click(on_play)
display(widgets.HBox([frame_input, play_button]), out)

Output()